# 现代方法实现线性回归

## 生成数据集

和上一节一样

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

提示一下，函数调用的时候参数名字前面加上 `*` 表示这个参数（一般为 list）作为函数的可变参数的对象

[具体笔记](https://www.notion.so/jackbrains/adb2e66d54b540a48506d76b37205450)

In [3]:
def Load_array(data_arrays, batch_size, is_train=True): #@save
    """Create a PyTorch iterator"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

# Create a instance of the iterator
batch_size = 10
data_iter = Load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 1.2135, -0.3331],
         [-0.9874,  2.2068],
         [-1.3478,  3.5137],
         [ 1.2499, -0.5261],
         [ 2.6171, -0.2414],
         [-0.9262,  0.3808],
         [-0.9328, -0.6382],
         [ 0.5212, -0.1983],
         [-0.1756, -0.5364],
         [ 0.2565,  1.5012]]),
 tensor([[  7.7916],
         [ -5.2820],
         [-10.4311],
         [  8.4765],
         [ 10.2404],
         [  1.0531],
         [  4.5043],
         [  5.9311],
         [  5.6606],
         [ -0.3855]])]

## 定义模型

对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 我们首先定义一个模型变量net，它是一个Sequential类的实例。Sequential类将多个层串联在一起。当给定输入数据时，Sequential实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，以此类推。在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。

在PyTorch中，**全连接层**在Linear类中定义。值得注意的是，我们将两个参数传递到 `nn.Linear` 中。第一个指定输入特征形状，即 2，第二个指定输出特征形状，输出特征形状为单个标量，因此为 1。

In [4]:
from torch import nn

# 2 means there are two layers in the net
# 1 means there are 1 output
net = nn.Sequential(nn.Linear(2, 1))

## 初始化模型参数

正如我们在构造 `nn.Linear` 时指定输入和输出尺寸一样，现在我们能直接访问参数以设定它们的初始值。 我们通过 `net[0]` 选择网络中的第一个图层，然后使用 `weight.data` 和 `bias.data` 方法访问参数。我们还可以使用替换方法 `normal_` 和 `fill_` 来重写参数值。

In [6]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义 Cost function

In [11]:
loss = nn.MSELoss()
# loss = nn.HuberLoss()

## 定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch 在 optim 模块中实现了该算法的许多变种。当我们实例化一个 SGD 实例时，我们要指定优化的参数（可通过 `net.parameters()` 从我们的模型中获得）以及优化算法所需的超参数字典。小批量随机梯度下降只需要设置 `lr` 值，这里设置为0.03。

In [8]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

通过深度学习框架的高级API来实现我们的模型只需要相对较少的代码。我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。当我们需要更复杂的模型时，高级API的优势将大大增加。当我们有了所有的基本组件，训练过程代码与我们从零开始实现时所做的非常相似。

In [12]:
num_epoch = 3

for epoch in range(num_epoch):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step() # 进行一次迭代
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000050
epoch 2, loss 0.000050
epoch 3, loss 0.000050


In [13]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0003,  0.0001])
b的估计误差： tensor([3.5286e-05])


In [17]:
# 得到梯度的值
net[0].weight.grad, net[0].bias.grad

(tensor([[-0.0032, -0.0026]]), tensor([-0.0010]))